<a href="https://colab.research.google.com/github/jeekyung/Review/blob/main/%5BFIFA_%EC%84%A0%EC%88%98_%EC%9D%B4%EC%A0%81%EB%A3%8C_%EC%98%88%EC%B8%A1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[베이스라인 코드]**

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/FIFA_train.csv')
test = pd.read_csv('/content/drive/MyDrive/FIFA_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/submission (1).csv')

In [64]:
train.head()

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0
2,7,L. Suárez,31,south america,2021,ST,right,5.0,91,91,3.0,80000000.0
3,8,Sergio Ramos,32,europe,2020,DF,right,4.0,91,91,3.0,51000000.0
4,9,J. Oblak,25,europe,2021,GK,right,3.0,90,93,1.0,68000000.0


In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8932 non-null   int64  
 1   name              8932 non-null   object 
 2   age               8932 non-null   int64  
 3   continent         8932 non-null   object 
 4   contract_until    8932 non-null   object 
 5   position          8932 non-null   object 
 6   prefer_foot       8932 non-null   object 
 7   reputation        8932 non-null   float64
 8   stat_overall      8932 non-null   int64  
 9   stat_potential    8932 non-null   int64  
 10  stat_skill_moves  8932 non-null   float64
 11  value             8932 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 837.5+ KB


* no NA

**[간단한 전처리]**

In [66]:
# 불필요한 칼럼 제거
train.drop(['id', 'name'], axis = 1, inplace=True)
test.drop(['id', 'name'], axis = 1, inplace = True)

In [67]:
train.contract_until.value_counts()

2019            2366
2021            2308
2020            2041
2022             761
2023             506
Jun 30, 2019     501
2018             327
Dec 31, 2018      64
May 31, 2019      19
2024              12
Jan 31, 2019      10
Jun 30, 2020       9
2025               3
Jan 1, 2019        2
2026               1
May 31, 2020       1
Jan 12, 2019       1
Name: contract_until, dtype: int64

In [68]:
#contract_until 변수 int형으로 변환
# 계약 연도만 추출
def func(string:object) -> int:
  # 계약 연도만 추출하여 int로 반환
  string = string[-4:]
  return int(string)

train['contract_until'] = train['contract_until'].apply(func)
test['contract_until'] = test['contract_until'].apply(func)

* def에서 :와 ->는 주석 설명. 위의 코드는 def func(string)과 같음.

In [69]:
# 문자형 변수 label 인코딩 진행
train['continent'] = train['continent'].astype('category').cat.codes
test['continent'] = test['continent'].astype('category').cat.codes

train.head()

,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,31,4,2021,ST,left,5.0,94,94,4.0,110500000.0
1,27,2,2020,GK,right,4.0,91,93,1.0,72000000.0
2,31,4,2021,ST,right,5.0,91,91,3.0,80000000.0
3,32,2,2020,DF,right,4.0,91,91,3.0,51000000.0
4,25,2,2021,GK,right,3.0,90,93,1.0,68000000.0


In [70]:
# 사이킷런 이용하여 label 인코딩도 가능
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train['position'])
train['position'] = encoder.transform(train['position'])
encoder.fit(test['position'])
test['position'] = encoder.transform(test['position'])

test.head()

,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,33,2,2022,3,right,5.0,94,94,5.0
1,26,4,2022,3,right,5.0,92,93,5.0
2,27,2,2023,2,right,4.0,91,92,4.0
3,27,2,2020,3,right,4.0,91,91,4.0
4,32,2,2020,2,right,4.0,91,91,4.0


In [71]:
train['prefer_foot'] = train['prefer_foot'].astype('category').cat.codes
test['prefer_foot'] = test['prefer_foot'].astype('category').cat.codes

In [72]:
#string형 변수 변환 완료
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               8932 non-null   int64  
 1   continent         8932 non-null   int8   
 2   contract_until    8932 non-null   int64  
 3   position          8932 non-null   int64  
 4   prefer_foot       8932 non-null   int8   
 5   reputation        8932 non-null   float64
 6   stat_overall      8932 non-null   int64  
 7   stat_potential    8932 non-null   int64  
 8   stat_skill_moves  8932 non-null   float64
 9   value             8932 non-null   float64
dtypes: float64(3), int64(5), int8(2)
memory usage: 575.8 KB


**[모델링]**

In [73]:
from sklearn.ensemble import RandomForestRegressor

train_x = train.drop('value', axis=1)
train_y = train['value']

model = RandomForestRegressor()
model.fit(train_x, train_y)

RandomForestRegressor()

In [74]:
pred = model.predict(test)
pred

array([7.9755e+07, 7.8720e+07, 6.9910e+07, ..., 5.9800e+04, 4.9000e+04,
       4.9300e+04])

In [75]:
submission['value'] = pred
submission.head()

,id,value
0,1,79755000.0
1,2,78720000.0
2,4,69910000.0
3,5,70930000.0
4,6,62180000.0


In [76]:
submission.to_csv('/content/drive/MyDrive/fifa.csv', index = False)

In [77]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./dacon_submit_api-0.0.4-py3-none-any.whl
dacon-submit-api is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [78]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/content/drive/MyDrive/fifa.csv', 
'ccac46cb82a2f20d1ac04f6b5f7b8834675ee36d04e4da44e7396a8dad05aaee', 
'403557', 
'로지', 
'' )

exception occurs. 예외가 발생했습니다 403
